# Intro
Breaking up the code into reusable blocks.

Confluent Python attempt is the version to use, kafka-python is incompatible.
Also see the Quix consumer work.

# Confluent Python
The OSS Kafka Python library is unfortunately stale (public release 2020) and not great, so it's best to use Confluent's based on librdkafka.

[Docs](https://docs.confluent.io/kafka-clients/python/current/overview.html), and [Github](https://github.com/confluentinc/confluent-kafka-python).

I think that the offset_config doesn't always work, so added the shell command to run it.

In [ ]:
pip install confluent-kafka

In [ ]:
bootstrap_servers = "localhost:19092,"
topics = ["weather_data_demo"]
client_id = "myClientId"
consumer_group = "stuCG"
offset_config = "earliest"

In [ ]:
# shell script to reset offsets
kafka-consumer-groups.sh --bootstrap-server localhost:19092 --group stuCG --topic weather_data_demo --reset-offsets --to-earliest --execute

In [ ]:
# --- Blend of Aiven and Quix's demos ---

# -- Creating A Consumer ---

from confluent_kafka import DeserializingConsumer
import json
import sys

def json_serializer(msg, s_obj=None):
    # return json.loads(msg.decode('ascii'))
    return json.loads(msg)

conf = {
    'bootstrap.servers': bootstrap_servers,
    'client.id': client_id,
    'group.id': consumer_group,
    # 'security.protocol': 'SSL',
    # 'ssl.ca.location': '../sslcerts/ca.pem',
    # 'ssl.certificate.location': '../sslcerts/service.cert',
    # 'ssl.key.location': '../sslcerts/service.key', 
    'value.deserializer': json_serializer,
    # 'key.deserializer': json_serializer, # if key in JSON
    'auto.offset.reset': offset_config,
    }

consumer = DeserializingConsumer(conf)


# --- Running the consumer ---

running = True

try:
    consumer.subscribe(topics)
    print(f"Subscribed to topics: {topics}")

    while running:
        msg = consumer.poll(timeout=1.0)
        if msg is None: 
            print("Waiting for message...")
            continue

        if msg.error():
            if msg.error().code() == KafkaError._PARTITION_EOF:
                # End of partition event
                sys.stderr.write('%% %s [%d] reached end at offset %d\n' %
                                 (msg.topic(), msg.partition(), msg.offset()))
            elif msg.error():
                raise KafkaException(msg.error())
        else:
             print(f"{msg.partition()}:{msg.offset()}: "
                  f"k={msg.key()} "
                  f"v={msg.value()}")
finally:
    # Close down consumer to commit final offsets.
    consumer.close()    

In [ ]:
# --- Running the consumer ---

running = True

try:
    consumer.subscribe(["weather_data_demo"])

    while running:
        msg = consumer.poll(timeout=1.0)
        if msg is None: continue

        if msg.error():
            if msg.error().code() == KafkaError._PARTITION_EOF:
                # End of partition event
                sys.stderr.write('%% %s [%d] reached end at offset %d\n' %
                                 (msg.topic(), msg.partition(), msg.offset()))
            elif msg.error():
                raise KafkaException(msg.error())
        else:
            print ("%d:%d: k=%s v=%s" % (
                msg.partition(),
                msg.offset(),
                msg.key(),
                msg.value()))
finally:
    # Close down consumer to commit final offsets.
    consumer.close()    

# Kafka Python
From [kafka-python](https://github.com/dpkp/kafka-python).

Tried to get this working, think it is failing on the metadata API so it can't discover the broker.
Will get a `NoBrokersAvailable` error so moved to Confluent python libray.

In [ ]:
pip install kafka-python

In [ ]:
from kafka import KafkaAdminClient
from kafka.admin import NewTopic
import json
import sys

In [ ]:
hostname = "localhost"
port = "19092"
topics = ["weather_data_demo"]

In [ ]:
# Create the consumer
from kafka import KafkaConsumer

'''
I think there's an issue with the metadata API call and the `kafka` library.
No brokers available I think it's failing on that.
'''

def json_serialiser(msg, s_obj):
    return json.loads(msg.decode('ascii'))

config = {
    'bootstrap_servers': hostname+":"+port,
    'client_id': 'myClient',
    'group_id': 'ConsumerAlpha',
    # 'security.protocol': 'SSL',
    # 'ssl.ca.location': '../sslcerts/ca.pem',
    # 'ssl.certificate.location': '../sslcerts/service.cert',
    # 'ssl.key.location': '../sslcerts/service.key', 
    'value_deserializer': json_serialiser,
    'key_deserializer': json_serialiser
}

# consumer = KafkaConsumer(config)
c2 = KafkaConsumer("weather_data_demo")

In [ ]:
running = True

In [ ]:
# Running the consumer

try:
    consumer.subscribe((topics))

    while running:
        msg = consumer.poll(timeout=1.0)
        if msg is None: 
            print("Waiting for message")
            continue
        if msg.error():
            if msg.error().code() == KafkaError._PARTITION_EOF:
                # End of partition event
                sys.stderr.write('%% %s [%d] reached end at offset %d\n' %
                                 (msg.topic(), msg.partition(), msg.offset()))
            elif msg.error():
                raise KafkaException(msg.error())
        else:
            # print ("%d:%d: k=%s v=%s" % (
            #     msg.partition(),
            #     msg.offset(),
            #     msg.key(),
            #     msg.value()))
            # test the udpate to f-strings
            print(f"{msg.partition()}:{msg.offset()}: "
                  f"k={msg.key()} "
                  f"v={msg.value()}")

finally:
    # Close down consumer to commit final offsets.
    consumer.close()   